In [1]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('qwen/Qwen2.5-32B-Instruct-GPTQ-Int4',cache_dir='./', revision='master')

2024-09-21 10:32:29,860 - modelscope - INFO - PyTorch version 2.3.1 Found.
2024-09-21 10:32:29,864 - modelscope - INFO - TensorFlow version 2.17.0 Found.
2024-09-21 10:32:29,865 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-09-21 10:32:29,930 - modelscope - INFO - Loading done! Current index file version is 1.9.5, with md5 50e04befe9e20553b47b2a2f8a6d1b72 and a total number of 945 components indexed
Downloading: 100%|█████████▉| 3.71G/3.71G [00:57<00:00, 68.7MB/s]
Downloading: 100%|█████████▉| 3.24G/3.24G [00:50<00:00, 68.7MB/s]
Downloading: 100%|██████████| 168k/168k [00:00<00:00, 1.69MB/s]
Downloading: 100%|██████████| 6.23k/6.23k [00:00<00:00, 8.98MB/s]
Downloading: 100%|██████████| 6.71M/6.71M [00:00<00:00, 18.8MB/s]
Downloading: 100%|██████████| 7.13k/7.13k [00:00<00:00, 9.11MB/s]
Downloading: 100%|██████████| 2.65M/2.65M [00:00<00:00, 11.6MB/s]


In [ ]:
!pip install modelscope==1.9.5 -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install "transformers>=4.44.0" -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install streamlit==1.24.0 -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install sentencepiece==0.1.99 -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install transformers_stream_generator==0.0.4 -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install datasets==2.13.0 -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install peft==0.10.0 -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install openai==1.42.0 -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install tqdm==4.64.1 -i https://pypi.tuna.tsinghua.edu.cn/simple/
# !pip install transformers==4.39.3 -i https://pypi.tuna.tsinghua.edu.cn/simple/
!python -m pip install setuptools==69.5.1 -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install vllm==0.5.3 -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install nest-asyncio -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install accelerate -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install tf-keras -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install autoawq
!pip install lmdeploy
!pip install loguru
!pip install optimum[exporters] auto-gptq==0.7.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/

[notice] A n

# 重启

In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-21 15:07:36.593553: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-21 15:07:36.626292: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-21 15:07:36.636406: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-21 15:07:36.658926: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to 

In [2]:
# 将JSONL文件转换为DataFrame
df = pd.read_csv('inference_data.csv',encoding='gbk')
ds = Dataset.from_pandas(df)
# op=['A','B','C','D','E']
def preprocess_data(ds):
    processed_data = []
    for sample in ds:
        problem = sample['problem']  # 获取当前行的 problem
        question=sample['question']
        options=sample['options']
        answer=sample['gpt_answer']
        inference=sample['gpt_inference']
        new_entry={
                'problem': problem,
                'question': question,
                'options': options,
                'answer': answer,
                'inference':inference
            }
        # for question in sample['questions']:  # 遍历每个 question
        #     # 给每个选项分配A, B, C等字母
        #     labeled_options = [
        #         f"{op[idx]}. {option}" for idx, option in enumerate(question['options'])
        #     ]
        #     new_entry = {
        #         'problem': problem,
        #         'question': question['question'],
        #         'options': str(labeled_options),
        #         'answer': question['answer']
        #     }
        processed_data.append(new_entry)
    return processed_data

# 假设 ds 是你获取的数据
preprocessed_data = preprocess_data(ds)

# 将预处理后的数据转换为 DataFrame
df = pd.DataFrame(preprocessed_data)
ds = Dataset.from_pandas(df)

In [3]:
tokenizer = AutoTokenizer.from_pretrained('./qwen/Qwen2.5-32B-Instruct-GPTQ-Int4', use_fast=False, trust_remote_code=True)
tokenizer

Qwen2Tokenizer(name_or_path='./qwen/Qwen2.5-32B-Instruct-GPTQ-Int4', vocab_size=151643, model_max_length=131072, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_w

In [4]:
def process_func(example):
    MAX_LENGTH = 1800    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    
    instruction = tokenizer(
    f"""
    <|im_start|>请按以下步骤逐步回答问题：
    一、仔细阅读以下材料。
    二、基于材料内容，逐步进行逻辑推理。
    三、根据推理结果选择最合适的选项。<|im_end|>
    <|im_start|>
    *材料*：{example['problem']}
    *问题*：{example['question']}
    *选项*：{example['options']}
    
    注意事项：
    1. 请严格按照问题要求回答，不能修改或篡改问题中的任何条件或表述。如果问题中有特定要求（如“通过中转点”），请严格遵守所有限定条件。
    2. 推理过程遵循闭世界假设，未观测到的事实或无法推断的事实视为假。只能依据材料中的信息进行推理，不得使用材料外的知识（如语言常识、常识推理等）。
    3. 请严格按以下 JSON 格式回复：{{"inference":"你的逻辑推理部分","answer":"A"}}。确保 JSON 格式完全合法，任何不符合该格式的回复视为错误。<|im_end|>
    <|im_start|>assistant\n
    """,
    add_special_tokens=False)

    
    # 模型的回答部分
    # response = tokenizer(f"{example['inference']}+'\n'+{example['answer']}", add_special_tokens=False)
    response = tokenizer(f'{{"inference": "{example["inference"]}", "answer": "{example["answer"]}"}}', add_special_tokens=False)
    # 将 instruction 和 response 结合起来，注意要加上结束符号
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # eos token 用 attention_mask 关注
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
    
    # 做一个截断以防输入过长
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }


In [5]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1418
})

In [6]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'\n    <|im_start|>请按以下步骤逐步回答问题：\n    一、仔细阅读以下材料。\n    二、基于材料内容，逐步进行逻辑推理。\n    三、根据推理结果选择最合适的选项。<|im_end|>\n    <|im_start|>\n    *材料*：有一个英文到法文的词汇表，包含以下对应词汇：\n\n1. the -> le\n2. cat -> chat\n3. jumps -> sauts\n4. over -> sur\n5. moon -> lune\n6. cow -> vache\n7. plays -> jouer\n8. fiddle -> violon\n9. egg -> bougre\n10. falls -> des chutes\n11. off -> de\n12. wall -> mur\n\n根据这个词汇表，翻译以下英文句子成法文：\n    *问题*：选择题 1：\n英文句子 "the cat jumps over the moon" 翻译成法文是：\n    *选项*：[\'A. le chat saute sur la lune\', \'B. le chat sauts sur le lune\', \'C. le sauts chat sur le lune\', \'D. le chat sauts sur le lune\']\n    \n    注意事项：\n    1. 请严格按照问题要求回答，不能修改或篡改问题中的任何条件或表述。如果问题中有特定要求（如“通过中转点”），请严格遵守所有限定条件。\n    2. 推理过程遵循闭世界假设，未观测到的事实或无法推断的事实视为假。只能依据材料中的信息进行推理，不得使用材料外的知识（如语言常识、常识推理等）。\n    3. 请严格按以下 JSON 格式回复：{"inference":"你的逻辑推理部分","answer":"A"}。确保 JSON 格式完全合法，任何不符合该格式的回复视为错误。<|im_end|>\n    <|im_start|>assistant\n\n    {"inference": "根据材料中的词汇表，首先我们进行逐词翻译：\n1. the -> le\n2. cat -> chat\n3. jumps -> sauts\

In [7]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[0]["labels"])))

'{"inference": "根据材料中的词汇表，首先我们进行逐词翻译：\n1. the -> le\n2. cat -> chat\n3. jumps -> sauts\n4. over -> sur\n5. the -> le\n6. moon -> lune\n将这些单词顺序拼接起来，得到的翻译是 \'le chat sauts sur le lune\'。所以最合适的选项是 \'D. le chat sauts sur le lune\'。", "answer": "D"}<|endoftext|>'

In [8]:
# !pip install torchvision==0.18.1
# !pip install torch==2.3.1
# !pip install transformers==4.43.2
# !pip install openai==1.42.0
# !pip install langchain_huggingface
# !pip install lmdeploy==0.6.0a0
# !pip install vllm==0.5.3


In [9]:

model_name='./qwen/Qwen2.5-32B-Instruct-GPTQ-Int4'
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto"
)
model

CUDA extension not installed.
CUDA extension not installed.
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:4713: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 5/5 [00:06<00:00,  1.32s/it]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 5120)
    (layers): ModuleList(
      (0-63): 64 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (rotary_emb): Qwen2RotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): Qwen2MLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): Qwen2RMSNorm((5120,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((5120,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((5120,), eps=1e-06)
  )
  (lm_head): Linear(in_features=5120, out_features=152064, bias=False)
)

In [10]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [11]:
model.dtype

torch.float16

In [12]:
#确认模型结构
for name, param in model.named_parameters():
    print(name)


model.embed_tokens.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.2.input_layernorm.weight
model.layers.2.post_attention_layernorm.weight
model.layers.3.input_layernorm.weight
model.layers.3.post_attention_layernorm.weight
model.layers.4.input_layernorm.weight
model.layers.4.post_attention_layernorm.weight
model.layers.5.input_layernorm.weight
model.layers.5.post_attention_layernorm.weight
model.layers.6.input_layernorm.weight
model.layers.6.post_attention_layernorm.weight
model.layers.7.input_layernorm.weight
model.layers.7.post_attention_layernorm.weight
model.layers.8.input_layernorm.weight
model.layers.8.post_attention_layernorm.weight
model.layers.9.input_layernorm.weight
model.layers.9.post_attention_layernorm.weight
model.layers.10.input_layernorm.weight
model.layers.10.post_attention_layernorm.weight
model.layers.11.input_layernorm.weigh

In [13]:
print(model)


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 5120)
    (layers): ModuleList(
      (0-63): 64 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (rotary_emb): Qwen2RotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): Qwen2MLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): Qwen2RMSNorm((5120,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((5120,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((5120,), eps=1e-06)
  )
  (lm_head): Linear(in_features=5120, out_features=152064, bias=False)
)


In [14]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # 自回归语言模型
    target_modules=[
        "self_attn.q_proj",    # 自注意力机制的 Query 权重
        "self_attn.k_proj",    # 自注意力机制的 Key 权重
        "self_attn.v_proj",    # 自注意力机制的 Value 权重
        "self_attn.o_proj",    # 自注意力机制的 Output 权重
        "mlp.gate_proj",       # 前馈网络的 gate 投影层
        "mlp.up_proj",         # 前馈网络的上升投影层
        "mlp.down_proj"        # 前馈网络的下降投影层
    ],
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'mlp.up_proj', 'self_attn.o_proj', 'self_attn.v_proj', 'mlp.gate_proj', 'mlp.down_proj', 'self_attn.q_proj', 'self_attn.k_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [15]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='./qwen/Qwen2.5-32B-Instruct-GPTQ-Int4', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'mlp.up_proj', 'self_attn.o_proj', 'self_attn.v_proj', 'mlp.gate_proj', 'mlp.down_proj', 'self_attn.q_proj', 'self_attn.k_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [16]:
model.print_trainable_parameters()

trainable params: 67,108,864 || all params: 1,624,904,704 || trainable%: 4.1300184456848


In [17]:
args = TrainingArguments(
    output_dir="./output/qwen2.5",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=1,
    save_steps=100, # 为了快速演示，这里设置10，建议你设置成100
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [18]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [19]:
import torch
torch.backends.cuda.enable_mem_efficient_sdp(False)

In [20]:
# 禁用模型缓存
# model.config.use_cache = False
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,0.822600
20,0.635400
30,0.668800
40,0.717700
50,0.590800
60,0.666500
70,0.698600
80,0.614400
90,0.630400
100,0.603700


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./qwen/Qwen2.5-32B-Instruct-GPTQ-Int4 - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./qwen/Qwen2.5-32B-Instruct-GPTQ-Int4 - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.u

TrainOutput(global_step=354, training_loss=0.6393436166526234, metrics={'train_runtime': 5681.3744, 'train_samples_per_second': 0.25, 'train_steps_per_second': 0.062, 'total_flos': 3743365583892480.0, 'train_loss': 0.6393436166526234, 'epoch': 0.998589562764457})

# 重启

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = './qwen/Qwen2.5-32B-Instruct-GPTQ-Int4'
lora_path = './output/qwen2.5/checkpoint-300' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.float16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

prompt = '''请按以下步骤逐步回答问题：
    
    一、仔细阅读以下材料。
    二、基于材料内容，逐步进行逻辑推理。
    三、根据推理结果选择最合适的选项。
    题目如下：\n\n### 题目:\n假设您需要构建一个二叉搜索树，其中每个节点或者是一个空的节点（称为"空节点"），或者是一个包含一个整数值和两个子树的节点（称为"数值节点"）。以下是构建这棵树的规则：\n\n1. 树中不存在重复的元素。\n2. 对于每个数值节点，其左子树的所有值都小于该节点的值，其右子树的所有值都大于该节点的值。\n3. 插入一个新值到一个"空节点"时，该"空节点"会被一个包含新值的新的数值节点取代。\n4. 插入一个已存在的数值将不会改变树。\n\n请基于以上规则，回答以下选择题：\n\n### 问题:\n选择题 1：\n给定一个空的二叉搜索树，插入下列数字: [5, 9, 2, 10, 11, 3]，下面哪个选项正确描述了结果树的结构？\nA. tree(5, tree(2, tree(3, nil, nil), nil), tree(9, tree(10, nil, nil), tree(11, nil, nil)))\nB. tree(5, tree(2, nil, tree(3, nil, nil)), tree(9, nil, tree(10, nil, tree(11, nil, nil))))\nC. tree(5, tree(3, tree(2, nil, nil), nil), tree(9, nil, tree(10, tree(11, nil, nil), nil)))\nD. tree(5, nil, tree(2, nil, tree(3, nil, nil)), tree(9, tree(11, nil, nil), tree(10, nil, nil)))
    注意事项：
    1. 请严格按照问题要求回答，不能修改或篡改问题中的任何条件或表述。如果问题中有特定要求（如“通过中转点”），请严格遵守所有限定条件。
    2. 推理过程遵循闭世界假设，未观测到的事实或无法推断的事实视为假。只能依据材料中的信息进行推理，不得使用材料外的知识（如语言常识、常识推理等）。
    3. 请严格按以下 JSON 格式回复：{{"inference":"你的逻辑推理部分","answer":"A"}}。确保 JSON 格式完全合法，任何不符合该格式的回复视为错误。
    '''
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "你是一个逻辑推理专家，擅长解决逻辑推理问题。"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 1024, "do_sample": True, "top_k": 5}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
# def api_retry(query, max_retries=5, retry_delay=5):
#     attempts = 0
#     outputs = None  
#     while attempts < max_retries:
#         try:
#             outputs = llm.generate([query], sampling_params, use_tqdm=False)
#             return outputs[0].outputs[0].text
#         except Exception as e:
#             attempts += 1
#             logger.error(f"推理失败: {e}. 正在重试 ({attempts}/{max_retries})")
#             time.sleep(retry_delay)
#     logger.error("推理多次失败，跳过此条输入")
#     return None
# messages = [
#                 {"role": "system", "content": "你是一名逻辑推理专家。你的任务是帮助用户分析逻辑问题并给出正确答案"},
#                 {"role": "user", "content": prompt}
#             ]
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True
# )
# res1 = api_retry(text)

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-21 16:44:58.512363: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-21 16:44:58.545069: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-21 16:44:58.555066: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-21 16:44:58.577892: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to 

{"inference": "首先，我们需要根据二叉搜索树的构建规则来逐步插入数字并构建树。按照给定顺序插入数字[5, 9, 2, 10, 11, 3]：

1. 插入5：tree(5, nil, nil)
2. 插入9：tree(5, nil, tree(9, nil, nil))
3. 插入2：tree(5, tree(2, nil, nil), tree(9, nil, nil))
4. 插入10：tree(5, tree(2, nil, nil), tree(9, nil, tree(10, nil, nil)))
5. 插入11：tree(5, tree(2, nil, nil), tree(9, nil, tree(10, nil, tree(11, nil, nil))))
6. 插入3：tree(5, tree(2, nil, tree(3, nil, nil)), tree(9, nil, tree(10, nil, tree(11, nil, nil))))

最终的树结构为：tree(5, tree(2, nil, tree(3, nil, nil)), tree(9, nil, tree(10, nil, tree(11, nil, nil))))，因此答案是B。", "answer": "B"}


In [ ]:
new_model_directory = "./merged_model_an"
merged_model = model.merge_and_unload()
# 将权重保存为safetensors格式的权重, 且每个权重文件最大不超过2GB(2048MB)
merged_model.save_pretrained(new_model_directory, max_shard_size="2048MB", safe_serialization=True)